In [9]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import seaborn as sns
import matplotlib.pyplot as plt

mids_dir = Path("D:\\MIDS-W207")
data = mids_dir/"datasets/soccertrack_square"
project = mids_dir/"MIDS-W207-Spring24-Soccer-Detection"
analysis = project/"analysis"

# Author: Timothy Majidzadeh
# Date Created: April 9, 2024
# Date Updated: April 9, 2024
# Description: Try CNN for classification on the soccertrack_square dataset.
# Re-uses some code from T. Majidzadeh's Homework 10 submission for MIDS W207 Spring 2024.
# Notes: [v1] Created program. 
# Inputs: Frame-by-frame labels saved as separate text files.
# Outputs: A CNN classifier for the soccertrack_square dataset.

In [10]:
# 1. Load the labels data and map to classes.
class_dict = {
    0: "No Objects",
    1: "Team 0 Only",
    2: "Team 1 Only",
    3: "Ball Only",
    4: "Team 0 and Team 1",
    5: "Team 0 and Ball",
    6: "Team 1 and Ball",
    7: "All Classes"
}

# Because certain class combinations are rare, it may be more effective to try each class individually.
# Remap to a value of 1 if one class is present, 0 otherwise.
ball_remap = {
    0: 0,
    1: 0,
    2: 0,
    3: 1,
    4: 0,
    5: 1,
    6: 1,
    7: 1
}

team_0_remap = {
    1: 1,
    2: 0,
    3: 0,
    4: 1,
    5: 1,
    6: 0,
    7: 1
}

team_1_remap = {
    1: 0,
    2: 1,
    3: 0,
    4: 1,
    5: 0,
    6: 1,
    7: 1
}
objects_per_image = pd.read_pickle(data/"objects_per_image_oversampled.pkl")

In [11]:
def class_mapping(ball_count, team_0_count, team_1_count):
    '''
    Assigns each row in the objects_per_image DataFrame to a class based on object counts.
    '''
    if (ball_count == 0) & (team_0_count == 0) & (team_1_count == 0):
        return 0
    elif (ball_count == 0) & (team_0_count > 0) & (team_1_count == 0):
        return 1
    elif (ball_count == 0) & (team_0_count == 0) & (team_1_count > 0):
        return 2
    elif (ball_count > 0) & (team_0_count == 0) & (team_1_count == 0):
        return 3
    elif (ball_count == 0) & (team_0_count > 0) & (team_1_count > 0):
        return 4
    elif (ball_count > 0) & (team_0_count > 0) & (team_1_count == 0):
        return 5
    elif (ball_count > 0) & (team_0_count == 0) & (team_1_count > 0):
        return 6
    else:
        return 7

In [15]:
objects_per_image['class'] = objects_per_image[
    ['img_ball_count', 'img_team_0_count', 'img_team_1_count']
].apply(lambda x: class_mapping(x.img_ball_count, x.img_team_0_count, x.img_team_1_count), axis=1)

objects_per_image['multiclass'] = objects_per_image['class']

objects_per_image.head()

class,image_name,img_ball_count,img_team_0_count,img_team_1_count,class,multiclass,ball_binary,team_0_binary,team_1_binary
23905,top_view_31511.png,1,4,4,7,7,1,1,1
17885,top_view_26094.png,1,2,2,7,7,1,1,1
84829,wide_view_38446.png,1,5,2,7,7,1,1,1
57622,wide_view_1396.png,1,9,9,7,7,1,1,1
88127,wide_view_41413.png,0,0,1,2,2,0,0,1


In [ ]:
# Re-used from T. Majidzadeh's HW 10 submission, with some edits for the use case.

def preprocess_data(image_paths, labels, splits):
    """ Split data into train, validation and test sets; apply transformaions and augmentations
    
    Params:
    -------
    image_paths (np.ndarray): Paths to load images.
    labels (np.ndarray): Labels of shape (N,)   
    splits (tuple): 3 values summing to 1 defining split of train, validation and test sets
    
    Returns:
    --------
    X_train (np.ndarray): Train images of shape (N_train, 224, 224, 3)
    y_train (np.ndarray): Train labels of shape (N_train,)
    X_val (np.ndarray): Val images of shape (N_val, 224, 224, 3)
    y_val (np.ndarray): Val labels of shape (N_val,)
    X_test (np.ndarray): Test images of shape (N_test, 224, 224, 3)
    y_test (np.ndarray): Test labels of shape (N_test,)
    
    """

    # Data is already shuffled on input.
    # create data splits (training, val, and test sets)
    train_val_index = int(np.floor(len(labels) * splits[0]))
    val_test_index = int(np.floor(len(labels) * (splits[0] + splits[1])))

    X_train_paths, y_train = image_paths[0:train_val_index], labels[0:train_val_index]
    X_val_paths, y_val = image_paths[train_val_index:val_test_index], labels[train_val_index:val_test_index]
    X_test_paths, y_test = image_paths[val_test_index:], labels[val_test_index:]

    X_train, X_val, X_test = [np.array([img_to_array(load_img(path, target_size=(400,400)) for path in paths]) for set in X_train_paths, X_val_paths, X_test_paths]
    
    # image augmentation (random flip) on training data
    X_train_augm = [tf.image.flip_left_right(image) for image in X_train]

    # concatenate original X_train and augmented X_train_augm data
    X_train = tf.concat([X_train, X_train_augm], axis=0)

    # concatenate y_train (note the label is preserved)
    y_train_augm = y_train
    y_train = tf.concat([y_train, y_train_augm],axis=0)

    # shuffle X_train and y_train, i.e., shuffle two tensors in the same order
    shuffle = tf.random.shuffle(tf.range(tf.shape(X_train)[0], dtype=tf.int32))
    X_train = tf.gather(X_train, shuffle).numpy() # transform X back to numpy array instead of tensor
    y_train = tf.gather(y_train, shuffle).numpy() # transform y back to numpy array instead of tensor

    # rescale training, val, and test images by dividing each pixel by 255.0 
    X_train, X_val, X_test = X_train / 255, X_val / 255, X_test / 255
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [7]:
# 2. Load the images and match to labels. Format as NumPy arrays.
paths = np.array([data/"images"/name for name in objects_per_image['image_name'][0:100]])
labels = np.array(objects_per_image['multiclass'][0:100])
splits = (0.8, 0.1, 0.1)
X_train, multiclass_train, X_val, multiclass_val, X_test, multiclass_test = preprocess_data(paths, labels, splits)

In [9]:
for object in [X_train, multiclass_train, X_val, multiclass_val, X_test, multiclass_test]:
    print(object.shape)

(80, 400, 400)

In [ ]:
# define an instance of the early_stopping class - from HW 10.
early_stopping = tf.keras.callbacks.EarlyStopping(
monitor='accuracy', 
verbose=1,
patience=4,
mode='max',
restore_best_weights=True)

In [14]:
tf.random.set_seed(1234)
np.random.seed(1234)

# initialize model
multiclass_model = tf.keras.Sequential()

# add convolutional layer to model1
inputs = tf.keras.layers.Input(shape=(400,400,3))
x = tf.keras.layers.Conv2D(
    filters = 64,
    kernel_size = (6, 6),
    strides = (2, 2),
    padding = 'same',
    data_format = 'channels_last',
    name = 'conv_1',
    activation = 'relu'
)
multiclass_model.add(inputs)
# Adding additiona complexity via more convolutional layers. The last one takes 2 strides instead of 1.
multiclass_model.add(x)
x = tf.keras.layers.Conv2D(
    filters = 64,
    kernel_size = (6, 6),
    strides = (1, 1),
    padding = 'same',
    data_format = 'channels_last',
    name = 'conv_2',
    activation = 'relu'
)
multiclass_model.add(inputs)
multiclass_model.add(x)

# add max pooling layer to model1
x = tf.keras.layers.MaxPooling2D(
    pool_size = (2, 2)
)
multiclass_model.add(x)

# add dropout layer to model1
x = tf.keras.layers.Dropout(
    rate = .3
)
multiclass_model.add(x)

# add a flattening layer to model1
x = tf.keras.layers.Flatten()
multiclass_model.add(x)

# add the classification layer to model1
x = layers.Dense(units=5)
multiclass_model.add(x)

# build and compile model1
multiclass_model.build(input_shape=(None, 400, 400, 3))
multiclass_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# print model1 summary
multiclass_model.summary()

# train model1 on (X_train, y_train) data
multiclass_model.fit(
    X_train,
    y_train,
    epochs=20,
    callbacks=[early_stopping]
)

In [ ]:
multiclass_model.evaluate(X_val, multiclass_val)

In [ ]:
multiclass_model.evaluate(X_test, multiclass_test)